# Data Preparation

In [98]:
import pandas as pd

In [99]:
df1 = pd.read_csv('/data3/animesh/vscode/Kaggle/auto_segmentation_scores.csv')
df1.head()

,imageid,auto_score
0,1_12_79_836_627.tif,0.342461
1,1_16_71_418_836.tif,0.207412
2,1_8_85_209_627.tif,0.148328
3,1_8_87_836_0.tif,0.299235
4,0_24_67_0_209.tif,0.208693


In [63]:
df1.shape

(1264, 2)

In [156]:
df2 = pd.read_csv('/data3/animesh/vscode/Kaggle/sam2_agreement_ratio_scores.csv', index_col= None)
df2.head()

,imageid,iou_score
0,1_11_86_0_836.png,0.962723
1,1_12_78_836_1044.png,0.987991
2,0_31_59_418_627.png,0.902557
3,1_19_79_627_836.png,0.494019
4,1_7_95_0_836.png,0.951294


In [154]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   imageid    5000 non-null   object 
 1   iou_score  5000 non-null   float64
dtypes: float64(1), object(1)
memory usage: 78.2+ KB


In [56]:
# Remove extensions from imageid in both DataFrames
df1['imageid'] = df1['imageid'].str.replace('.tif', '', regex=False)
df2['imageid'] = df2['imageid'].str.replace('.png', '', regex=False)

In [57]:
df2_filtered = df2[df2["imageid"].isin(df1['imageid'].values)]
df2_filtered

,imageid,iou_score
0,1_11_86_0_836,0.962723
1,1_12_78_836_1044,0.987991
2,0_31_59_418_627,0.902557
3,1_19_79_627_836,0.494019
4,1_7_95_0_836,0.951294
...,...,...
663,0_19_13_836_627,0.625275
664,0_15_3_627_209,0.956863
665,0_32_69_209_0,0.984070
666,1_25_78_836_836,0.039642


In [157]:
# Remove extensions from imageid in both DataFrames
df1['imageid'] = df1['imageid'].str.replace('.tif', '', regex=False)
df2['imageid'] = df2['imageid'].str.replace('.png', '', regex=False)

# Perform the left merge based on the cleaned imageid columns
df_combined = pd.merge(df1, df2, on="imageid", how="left")

# Create the 'combined_score' column by multiplying 'auto_score' and 'iou_score'
df_combined['combined_score'] = df_combined['auto_score'] * df_combined['iou_score']

# Display the resulting DataFrame
print(df_combined)


               imageid  auto_score  iou_score  combined_score
0      1_12_79_836_627    0.342461   0.806061        0.276044
1      1_16_71_418_836    0.207412   0.906876        0.188097
2       1_8_85_209_627    0.148328   0.934448        0.138604
3         1_8_87_836_0    0.299235   0.833801        0.249503
4        0_24_67_0_209    0.208693   0.993835        0.207406
...                ...         ...        ...             ...
2279  0_19_66_1044_836    0.101182   0.133347        0.013492
2280     1_12_79_418_0    0.333739   0.827881        0.276297
2281    1_13_83_0_1044    0.224008   0.913025        0.204524
2282     2_20_41_836_0    0.131080   0.967331        0.126798
2283      1_7_97_0_418    0.300350   0.926468        0.278264

[2284 rows x 4 columns]


In [ ]:
combined_score = df_combined['iou_score'] * (1 + 0.1 * df_combined['auto_score'])

In [158]:
# Remove extensions from imageid in both DataFrames
df1['imageid'] = df1['imageid'].str.replace('.tif', '', regex=False)
df2['imageid'] = df2['imageid'].str.replace('.png', '', regex=False)

# Perform the left merge based on the cleaned imageid columns
df_combined = pd.merge(df1, df2, on="imageid", how="left")

# Create the 'combined_score' column by multiplying 'auto_score' and 'iou_score'
df_combined['combined_score'] = 0.95* df_combined['iou_score'] + 0.2 * df_combined['auto_score']

# Display the resulting DataFrame
print(df_combined)

               imageid  auto_score  iou_score  combined_score
0      1_12_79_836_627    0.342461   0.806061        0.834250
1      1_16_71_418_836    0.207412   0.906876        0.903014
2       1_8_85_209_627    0.148328   0.934448        0.917391
3         1_8_87_836_0    0.299235   0.833801        0.851958
4        0_24_67_0_209    0.208693   0.993835        0.985882
...                ...         ...        ...             ...
2279  0_19_66_1044_836    0.101182   0.133347        0.146916
2280     1_12_79_418_0    0.333739   0.827881        0.853235
2281    1_13_83_0_1044    0.224008   0.913025        0.912175
2282     2_20_41_836_0    0.131080   0.967331        0.945180
2283      1_7_97_0_418    0.300350   0.926468        0.940214

[2284 rows x 4 columns]


In [166]:
import os
import numpy as np
import pandas as pd
from PIL import Image

# Paths to the folders containing the patches and labels
data_dir = "/data3/animesh/vscode/Kaggle/dataset"
patches_dir = os.path.join(data_dir, "training_patches")
labels_dir = os.path.join(data_dir, "training_noisy_labels")

# Assuming df_combined is already defined with an 'imageid' column containing the image names without extensions
# Convert the 'imageid' column to a set of filenames for efficient lookup
image_ids = set(df_combined['imageid'])

# Initialize lists to store patch and label images as numpy arrays
patches = []
labels = []

# Loop through the files in the patches directory
for filename in os.listdir(patches_dir):
    # Extract the filename without extension
    image_name = os.path.splitext(filename)[0]
    
    # Check if this image_name is in the image_ids set and the file is a .png
    if image_name in image_ids and filename.endswith(".png"):
        # Construct the full file paths
        patch_path = os.path.join(patches_dir, filename)
        label_path = os.path.join(labels_dir, filename)  # Same filename for labels
        
        # Check if the corresponding label exists
        if os.path.exists(label_path):
            # Read the patch and label images
            patch_img = np.array(Image.open(patch_path))
            label_img = np.array(Image.open(label_path))
            
            # Append them to the lists
            patches.append(patch_img)
            labels.append(label_img)

# Convert lists to NumPy arrays
images = np.array(patches)
masks = np.array(labels)

# Display the shapes of the arrays
print("Patches array shape:", images.shape)
print("Labels array shape:", masks.shape)


Patches array shape: (2284, 256, 256, 3)
Labels array shape: (2284, 256, 256)


In [167]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Custom Dataset Class
class ImageMaskScoreDataset(Dataset):
    def __init__(self, images, masks, scores, transform=None):
        self.images = images
        self.masks = masks
        self.scores = scores
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]
        score = self.scores[idx]

        # Normalize image
        if self.transform:
            image = self.transform(image)

        return torch.tensor(image, dtype=torch.float32).permute(2, 0, 1), \
               torch.tensor(mask, dtype=torch.float32).unsqueeze(0), \
               torch.tensor(score, dtype=torch.float32)

# Splitting dataset into train, val, and test
def create_dataloaders(images, masks, scores, batch_size=32, test_size=0.2, val_size=0.1):
    # Train-test split
    train_indices, test_indices, y_train, y_test = train_test_split(
        np.arange(len(images)), scores, test_size=test_size, random_state=42)

    # Further split train into train-validation
    train_size = int((1 - val_size) * len(train_indices))
    val_size = len(train_indices) - train_size
    train_indices, val_indices = train_indices[:train_size], train_indices[train_size:]

    
    
    transform = lambda x: (x / 255.0)

    # Creating datasets
    train_dataset = ImageMaskScoreDataset(images[train_indices], masks[train_indices], y_train[:train_size], transform=transform)
    val_dataset = ImageMaskScoreDataset(images[val_indices], masks[val_indices], y_train[train_size:], transform=transform)
    test_dataset = ImageMaskScoreDataset(images[test_indices], masks[test_indices], y_test, transform=transform)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader


In [168]:
# Example usage
train_loader, val_loader, test_loader = create_dataloaders(images, masks, df_combined['combined_score'].values, batch_size = 8)

In [169]:
# First batch from the train_loader
for images, masks, scores in train_loader:
    print(f"Images batch shape: {images.shape}")  # Shape of images in a batch
    print(f"Masks batch shape: {masks.shape}")    # Shape of masks in a batch
    print(f"Scores batch shape: {scores.shape}")  # Shape of scores in a batch
    break  # Check only the first batch

Images batch shape: torch.Size([8, 3, 256, 256])
Masks batch shape: torch.Size([8, 1, 256, 256])
Scores batch shape: torch.Size([8])


# Model Architecture

In [105]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNetSmall(nn.Module):
    def __init__(self):
        super(UNetSmall, self).__init__()

        # Encoder with fewer channels
        self.enc_conv1 = self.double_conv(4, 16)  # Reduced channels
        self.enc_conv2 = self.double_conv(16, 32)
        self.enc_conv3 = self.double_conv(32, 64)
        self.enc_conv4 = self.double_conv(64, 128)

        # Decoder with corresponding channel reductions
        self.up_conv3 = self.up_conv(128, 64)
        self.dec_conv3 = self.double_conv(128, 64)
        self.up_conv2 = self.up_conv(64, 32)
        self.dec_conv2 = self.double_conv(64, 32)
        self.up_conv1 = self.up_conv(32, 16)
        self.dec_conv1 = self.double_conv(32, 16)

        # Adaptive pooling to match output shape for fully connected layer
        self.pool = nn.AdaptiveAvgPool2d((8, 8))  # Adjust to final output size as needed

        # Final output layer for score prediction
        self.final_fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 8 * 8, 64),  # Adjusted for reduced feature size from pooling
            nn.ReLU(),
            nn.Linear(64, 1)  # Single output for score prediction
        )

    def double_conv(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def up_conv(self, in_channels, out_channels):
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, images, masks):
        # Concatenate images and masks along channel dimension
        x = torch.cat((images, masks), dim=1)  # (B, 4, 256, 256)

        # Encoder
        e1 = self.enc_conv1(x)
        e2 = self.enc_conv2(F.max_pool2d(e1, 2))
        e3 = self.enc_conv3(F.max_pool2d(e2, 2))
        e4 = self.enc_conv4(F.max_pool2d(e3, 2))

        # Decoder
        d3 = self.up_conv3(e4)
        d3 = self.dec_conv3(torch.cat([d3, e3], dim=1))
        d2 = self.up_conv2(d3)
        d2 = self.dec_conv2(torch.cat([d2, e2], dim=1))
        d1 = self.up_conv1(d2)
        d1 = self.dec_conv1(torch.cat([d1, e1], dim=1))

        # Adaptive pooling to resize output for the fully connected layer
        d1_pooled = self.pool(d1)

        # Final fully connected layer to predict the score
        out = self.final_fc(d1_pooled)
        return out

# Instantiate the model
model = UNetSmall()


In [170]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNetSmallImproved(nn.Module):
    def __init__(self):
        super(UNetSmallImproved, self).__init__()

        # Encoder with BatchNorm and LeakyReLU
        self.enc_conv1 = self.double_conv(4, 32)  # Increased initial channels slightly
        self.enc_conv2 = self.double_conv(32, 64)
        self.enc_conv3 = self.double_conv(64, 128)
        self.enc_conv4 = self.double_conv(128, 256)

        # Decoder with corresponding channel scaling
        self.up_conv3 = self.up_conv(256, 128)
        self.dec_conv3 = self.double_conv(256, 128)
        self.up_conv2 = self.up_conv(128, 64)
        self.dec_conv2 = self.double_conv(128, 64)
        self.up_conv1 = self.up_conv(64, 32)
        self.dec_conv1 = self.double_conv(64, 32)

        # Adaptive pooling to match output shape for fully connected layer
        self.pool = nn.AdaptiveAvgPool2d((4, 4))  # Adjusted pooling size

        # Fully connected layers with dropout
        self.final_fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 4 * 4, 128),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(128, 1)
        )

        # Initialize weights
        self.apply(self.init_weights)

    def double_conv(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),  # Batch normalization
            nn.LeakyReLU(0.1, inplace=True),  # LeakyReLU activation
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.1, inplace=True)
        )

    def up_conv(self, in_channels, out_channels):
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, images, masks):
        # Concatenate images and masks along channel dimension
        x = torch.cat((images, masks), dim=1)  # (B, 4, 256, 256)

        # Encoder
        e1 = self.enc_conv1(x)
        e2 = self.enc_conv2(F.max_pool2d(e1, 2))
        e3 = self.enc_conv3(F.max_pool2d(e2, 2))
        e4 = self.enc_conv4(F.max_pool2d(e3, 2))

        # Decoder
        d3 = self.up_conv3(e4)
        d3 = self.dec_conv3(torch.cat([d3, e3], dim=1))
        d2 = self.up_conv2(d3)
        d2 = self.dec_conv2(torch.cat([d2, e2], dim=1))
        d1 = self.up_conv1(d2)
        d1 = self.dec_conv1(torch.cat([d1, e1], dim=1))

        # Adaptive pooling to resize output for the fully connected layer
        d1_pooled = self.pool(d1)

        # Final fully connected layer to predict the score
        out = self.final_fc(d1_pooled)
        return out

    @staticmethod
    def init_weights(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            nn.init.kaiming_normal_(m.weight, nonlinearity='leaky_relu')
        elif isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            nn.init.constant_(m.bias, 0)

# Instantiate the improved model
model = UNetSmallImproved()


In [171]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
from tqdm import tqdm
# Define optimizer and criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()
def train_model(model, train_loader, val_loader, optimizer = optimizer, criterion = criterion, num_epochs=10):
    # Set up learning rate scheduler (ReduceLROnPlateau)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        epoch_losses = []

        # Progress bar for the training phase
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images = batch[0].to(device)  # Assuming images are at index 0
            masks = batch[1].to(device)   # Assuming masks are at index 1
            scores = batch[2].to(device)  # Assuming scores are at index 2

            # Forward pass
            outputs = model(images, masks).squeeze()  # Squeeze the outputs to match the shape of scores

            # Compute loss
            loss = criterion(outputs, scores)

            # Backward pass (compute gradients of parameters w.r.t. loss)
            optimizer.zero_grad()  # Clear previous gradients
            loss.backward()  # Backpropagation to compute gradients

            # Optimize parameters (correctly using optimizer.step())
            optimizer.step()  # Update weights, avoid calling optimizer()

            # Record loss for the current batch
            epoch_losses.append(loss.item())

        # Print mean loss for the epoch
        mean_epoch_loss = sum(epoch_losses) / len(epoch_losses)
        print(f'EPOCH: {epoch+1}/{num_epochs}')
        print(f'Mean Train Loss: {mean_epoch_loss:.4f}')

        lr = optimizer.param_groups[0]["lr"]
        print(f'Learning_rate:{lr}')

        # Validation phase
        val_losses = []
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient calculation for validation
            for batch in val_loader:
                images = batch[0].to(device)
                masks = batch[1].to(device)
                scores = batch[2].to(device)

                # Forward pass
                outputs = model(images, masks).squeeze()

                # Compute validation loss
                val_loss = criterion(outputs, scores)
                val_losses.append(val_loss.item())

        mean_val_loss = sum(val_losses) / len(val_losses)
        print(f'Mean Validation Loss: {mean_val_loss:.4f}')

        # Step the learning rate scheduler based on validation loss
        scheduler.step(mean_val_loss)

    print("Training complete.")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Start training
train_model(model, train_loader, val_loader, num_epochs=10)


/data3/animesh/vscode/Kaggle/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/10:   0%|          | 0/206 [00:00<?, ?it/s]

Epoch 1/10: 100%|██████████| 206/206 [10:59<00:00,  3.20s/it]


EPOCH: 1/10
Mean Train Loss: 0.2301
Learning_rate:0.0001
Mean Validation Loss: 0.0426


Epoch 2/10: 100%|██████████| 206/206 [10:59<00:00,  3.20s/it]


EPOCH: 2/10
Mean Train Loss: 0.0811
Learning_rate:0.0001
Mean Validation Loss: 0.0465


...

EPOCH: 8/10
Mean Train Loss: 0.0600
Learning_rate:0.0001
Mean Validation Loss: 0.0372


Epoch 9/10: 100%|██████████| 206/206 [09:48<00:00,  2.86s/it]


EPOCH: 9/10
Mean Train Loss: 0.0613
Learning_rate:1e-05
Mean Validation Loss: 0.0356


Epoch 10/10: 100%|██████████| 206/206 [09:43<00:00,  2.83s/it]


EPOCH: 10/10
Mean Train Loss: 0.0589
Learning_rate:1e-05
Mean Validation Loss: 0.0387
Training complete.


# Prediction

In [172]:
# Paths to the folders containing the patches and labels
data_dir = "/data3/animesh/vscode/Kaggle/dataset"
patches_dir = os.path.join(data_dir, "training_patches")
labels_dir = os.path.join(data_dir, "training_noisy_labels")

# Function to load all images in a directory
def load_images_from_folder(folder):
    images = []
    filenames = os.listdir(folder)
    for filename in filenames:
        if filename.endswith(".png"):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path)#.convert('RGB')  # Converting to RGB for consistency
            img = np.array(img)  # Convert to numpy array
            images.append(img)
    return images, filenames

# Load training patches and noisy labels
patches, patch_filenames = load_images_from_folder(patches_dir)
noisy_labels, label_filenames = load_images_from_folder(labels_dir)

In [173]:
images = np.array(patches)
masks = np.array(noisy_labels)

In [109]:
import numpy as np
import torch


# Function to normalize the images using calculated mean and std
def apply_normalization(images):
    # Normalize the images (per channel)
    return (images / 255.0)

# Normalize the images based on the entire dataset
def prepare_normalized_data(images, masks):
   
    # Apply normalization to the images
    normalized_images = apply_normalization(images)

    # Convert images and masks to PyTorch tensors
    normalized_images = torch.tensor(normalized_images).permute(0, 3, 1, 2).float()  # [batch, channels, height, width]
    masks = torch.tensor(masks).unsqueeze(1).float()  # [batch, 1, height, width]

    return normalized_images, masks

In [174]:
normalized_images, masks = prepare_normalized_data(images, masks)

In [56]:
from torch.utils.data import DataLoader, TensorDataset

# Function to create a DataLoader
def create_dataloader(images, masks, batch_size=32, shuffle=False):
    # Create a TensorDataset with images and masks
    dataset = TensorDataset(images, masks)
    
    # Create a DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    
    return dataloader


In [175]:
# Assume normalized_images and normalized_masks are prepared from the previous step
dataloader = create_dataloader(normalized_images, masks, batch_size=32, shuffle=False)

In [176]:
import torch
import pandas as pd

# Function to predict scores and store them in a DataFrame
def predict_and_store_scores(model, dataloader, patch_filenames, device):
    model.eval()  # Set model to evaluation mode
    predictions = []
    
    # Iterate through the dataloader and predict scores
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            images = batch[0].to(device)  # Load images to the device
            masks = batch[1].to(device)   # Load masks to the device
            
            # Ensure that both images and masks have the same batch size
            if images.shape[0] != masks.shape[0]:
                raise ValueError(f"Batch size mismatch: images batch {images.shape[0]} and masks batch {masks.shape[0]}")
            
            # Predict scores
            predicted_scores = model(images, masks)  # Assuming model takes both images and masks
            predicted_scores = predicted_scores.squeeze(1)  # Ensure output is [batch_size, 1] and squeeze to [batch_size]
            
            # Move predicted scores to CPU and convert to NumPy
            predicted_scores = predicted_scores.cpu().numpy()
            predictions.extend(predicted_scores)

    # Create a DataFrame to store imageid and iou_score
    df_predictions = pd.DataFrame({
        'imageid': patch_filenames,  # Image IDs from patch_filenames
        'iou_score': predictions      # Predicted IoU scores
    })

    return df_predictions

# Example usage:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming model and dataloader are prepared, and patch_filenames is a list of filenames
df_results = predict_and_store_scores(model, dataloader, patch_filenames, device)

# Save the DataFrame to a CSV file if needed
# df_results.to_csv('predicted_iou_scores.csv', index=False)

# Print the first few rows to verify
print(df_results.head())


                imageid  iou_score
0     1_11_86_0_836.png   0.816832
1  1_12_78_836_1044.png   0.779570
2   0_31_59_418_627.png   0.815308
3   1_19_79_627_836.png   0.805756
4      1_7_95_0_836.png   0.823522


In [177]:
# Sort the DataFrame by IoU score (ascending order)
df_sorted = df_results.sort_values(by='iou_score', ascending=False)

# Print the sorted DataFrame
print(df_sorted.head())

                 imageid  iou_score
1733   1_14_72_0_627.png   1.077527
1519  1_7_87_627_418.png   1.077338
3281  1_6_87_836_418.png   1.056382
3828    1_8_86_209_0.png   1.054950
1937  1_7_87_627_627.png   1.045420


In [178]:
imageid = list(df_sorted['imageid'])
submission = pd.DataFrame({'id' : range(5000), 'imageid': imageid }, index = None)
submission.head()

,id,imageid
0,0,1_14_72_0_627.png
1,1,1_7_87_627_418.png
2,2,1_6_87_836_418.png
3,3,1_8_86_209_0.png
4,4,1_7_87_627_627.png


In [179]:
# Save the DataFrame to a CSV file if needed
submission.to_csv('/data3/animesh/vscode/Kaggle/submission_33.csv', index=False)

# Rough work

In [65]:
print(np.isnan(patches_array).any(), np.isnan(labels_array).any())
print(np.isinf(patches_array).any(), np.isinf(labels_array).any())
print(np.isnan(df_combined['combined_score'].values).any(), np.isnan(df_combined['combined_score'].values).any())

False False
False False
False False


In [31]:
# Select only numeric columns to check for NaN or Inf
numeric_df = df_combined.select_dtypes(include=[np.number])

# Find rows with NaN values
nan_rows = df_combined[numeric_df.isna().any(axis=1)]

# Find rows with Inf values (only on numeric columns)
inf_rows = df_combined[(np.isinf(numeric_df)).any(axis=1)]

# Combine the rows with NaN or Inf values for easier inspection
nan_or_inf_rows = pd.concat([nan_rows, inf_rows]).drop_duplicates()

# Display rows with NaN or Inf values
print("Rows with NaN or Inf values:")
print(nan_or_inf_rows)


Rows with NaN or Inf values:
              imageid  auto_score  iou_score  combined_score
25    0_34_62_836_836    0.337333        NaN             NaN
39      2_19_45_418_0    0.215769        NaN             NaN
70   1_12_80_1044_627    0.437527        NaN             NaN
78   2_15_45_1044_836    0.244235        NaN             NaN
84     1_15_74_1044_0    0.388788        NaN             NaN
155   0_30_63_209_836    0.335499        NaN             NaN
171     2_26_34_0_836    0.343716        NaN             NaN
207   1_12_79_418_836    0.403307        NaN             NaN
209    1_9_97_836_627    0.373431        NaN             NaN
211   0_32_58_418_209    0.269550        NaN             NaN
267  1_5_100_1044_418    0.426051        NaN             NaN
278    1_5_103_0_1044    0.397366        NaN             NaN
329   0_17_67_836_418    0.276497        NaN             NaN
351     1_21_79_836_0    0.351732        NaN             NaN
359   0_18_19_627_418    0.327528        NaN            

In [37]:
import pandas as pd

def find_nan_rows(df):
    """
    Function to find rows in a DataFrame that contain NaN values.

    Parameters:
    df (pd.DataFrame): Input DataFrame.

    Returns:
    pd.DataFrame: DataFrame containing rows with NaN values.
    """
    # Select rows with NaN values
    nan_rows = df[df.isna().any(axis=1)]
    
    return nan_rows

# Example usage:
nan_rows = find_nan_rows(df_combined)
print(nan_rows)


              imageid  auto_score  iou_score  combined_score
25    0_34_62_836_836    0.337333        NaN             NaN
39      2_19_45_418_0    0.215769        NaN             NaN
70   1_12_80_1044_627    0.437527        NaN             NaN
78   2_15_45_1044_836    0.244235        NaN             NaN
84     1_15_74_1044_0    0.388788        NaN             NaN
155   0_30_63_209_836    0.335499        NaN             NaN
171     2_26_34_0_836    0.343716        NaN             NaN
207   1_12_79_418_836    0.403307        NaN             NaN
209    1_9_97_836_627    0.373431        NaN             NaN
211   0_32_58_418_209    0.269550        NaN             NaN
267  1_5_100_1044_418    0.426051        NaN             NaN
278    1_5_103_0_1044    0.397366        NaN             NaN
329   0_17_67_836_418    0.276497        NaN             NaN
351     1_21_79_836_0    0.351732        NaN             NaN
359   0_18_19_627_418    0.327528        NaN             NaN
397   1_7_87_418_1044   